# Introduction

These are my notes on statistics and quantitative methods for analysis in the social sciences.

## Why use statistics?

As social scientists and researchers we want to answer questions about the social world.
At its most simplistic (and I completely acknowledge that I'm massively generalising here), we usually seek to describe the world or quantify the world.
For these tasks we use qualitative methods and quantitative methods respectively.

To quantify the world we might want to know things like, "how many people have been a victim of crime?", "how many people have mental health problems?", or "how many people have social science degrees?".

Sometimes we have complete (or near--complete) data about everybody in a population.
For example, in the UK the decennial (i.e. every ten years) census is a count of everybody, including certain characteristics such as health, education, and employment.
In this case the population is everybody in the UK.

More often than not it is impractical to ask everybody our questions.
Usually the cost and time required to carry out such a survey prohibit asking every single person what we want to know, but also very few people would answer everything that they were asked (the only reason everybody responds to the census is because [it's a crime not to complete the census](https://en.wikipedia.org/wiki/Census_in_the_United_Kingdom#Criminal_law) and [people have been prosecuted for not completing it](https://www.theguardian.com/uk/2012/jan/27/120-convicted-census-forms-2011)).
Instead we take a *sample* of the population, and *infer*, from our sample, what the population is like.

For example, we might ask a random sample of 1,000 people what their favourite hot drink is.
It's not that we want to know what this 1,000 people think more than any other 1,000 people.
Instead, they are our *sample* and, based on their responses, we can *infer* what the most popular hot drink is for the *population*.
In the UK, the [most popular hot drink is tea](https://www.statista.com/statistics/697383/favorite-hot-drinks-united-kingdom-uk/), but everybody knows it should be coffee.

![Coffee is clearly superior to tea](../images/coffee.jpg)

Obtaining knowledge about a *population* by *inferring* from a *sample* is the cornerstone of quantitative social science research, and uses many statistical techniques to be able to do this.
The good news is the most difficult challenge is often deciding which technique to use; using the technique is often like following a recipe.


## Packages

I use `python3` to process and analyse the data, and a number of packages install through anaconda.


In [ ]:
import os
import requests
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.stats

## Data sources

These tutorials use a number of teaching data sets available from the [UK Data Service](https://www.ukdataservice.ac.uk/) and Nomisweb (FYI [Nomisweb have a great API](https://www.nomisweb.co.uk/api/v01/help) for reproducible research) under terms of the [Open Government License](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/):

> Office for National Statistics, University of Manchester, Cathie Marsh Institute for Social Research (CMIST), UK Data Service, 2016, Living Costs and Food Survey, 2013: Unrestricted Access Teaching Dataset, [data collection], Office for National Statistics, 2nd Edition, Office for National Statistics, [original data producer(s)]. Accessed 1 October 2018. SN: 7932, http://doi.org/10.5255/UKDA-SN-7932-2. Contains public sector information licensed under the Open Government Licence v2.0

<br />

> Office for National Statistics, 2014, 2011 Census. Accessed October 2018. Contains public sector information licensed under the Open Government Licence v2.0.

In [ ]:
os.makedirs("../data/external", exist_ok=True)

if not os.path.isfile("../data/external.food.zip"):
    food = "https://beta.ukdataservice.ac.uk/Umbraco/Surface/Discover/GetDownload?studyNumber=7932&fileName=7932tab_818dcb297393b2557b07f05acaae70b9.zip"
    food = requests.get(food)
    food = food.content
    outfile = open("../data/external/food.zip", "wb")
    outfile.write(food)

if not os.path.isdir("../data/external/UKDA-7932-tab/"):
    shutil.unpack_archive("../data/external/food.zip", extract_dir = "../data/external/")

In [ ]:
food = pd.read_csv(
    "../data/external/UKDA-7932-tab/tab/lcfs_2013_teaching.tab",
    sep = "\t"
)

# Check it's loaded correctly
food.head()

# Levels of measurement

Levels of measurement describe the nature of your data point.
They are important because they determine what statistical tests you can perform on them.
The four levels of measurement are:


## Nominal

Also known as **categorical**.
These are labels such as male/female; no religion/religion.
They have no inherent order; one response is not 'better' or 'higher' than another.
As social scientists you'll find these are pretty common.


## Ordinal

Similar to nominal in that they are labels (rather than numbers), but the labels have a rank or order.
For example, a 'strongly disagree' to 'strongly agree' scale is ordinal.
Another example would be 'guilty' and 'not guilty'.


## Interval

Interval data is numerical but does not have a meaningful zero value.
The most common example often given is temperature expressed in degrees celcius.
A temperature of 20&deg;C is not twice as hot as 10&deg;C, because 0&deg;C is arbitrary rather than based on the absence of heat.

A more common example in the social sciences is date.
Years are based on an arbitrary zero (0AD); months are based on twelve months that do not have a 'zero'; days do not have a zero and reset each month.


## Ratio

Ratio data is again numerical, but differs from interval because it has a meaningful, non--arbitrary, zero.
As we saw above dates are interval, but age in years is ratio because zero years old is the lowest age you can be.

It is more common for numerical data to be ratio than interval.
Other examples include income and number of events (e.g. number of crimes in an area).


# Central tendency

Measures of central tendency is a fancy phrase for 'average'.
They are a single data point used to represent a 'typical' value from your data.
Depending on your level of measurement you can use one or more measures of central tendency.

## Mode

The most common value.
Mode is the only measure of central tendency you can provide for [nominal data](../levels-of-measurement#nominal).

For example, the variable `A121r` in our food data set is of household tenure type.
The available options are:

1. public rented (i.e. rented from a council)
2. private rented (i.e. rented from a landlord)
3. owned

A frequency (count) table of this variable shows that `owned` is the most common type of tenure:

In [ ]:
food["A121r"] = food["A121r"].astype("category")
food["A121r"].cat.categories = ["public rented", "private rented", "owned"]
food["A121r"].value_counts()

## Median

The median is the 'middle' point.
It's only appropriate for ordered data (i.e. ordinal or numeric) and is calculated by arranging your data in order and selecting the mid--point.
`P344pr` is the gross normal weekly household income for each respondent. The following are incomes for the first five respondents as an example:

In [ ]:
food["P344pr"].head()

Note that these have been [top--coded](https://en.wikipedia.org/wiki/Top-coded), so for this example I remove the largest income, which is:

In [ ]:
food["P344pr"].max()

In [ ]:
food_trimmed = food[food["P344pr"] < 1184]
food_trimmed["P344pr"].max()

Which looks like this when we plot it as a distribution:

In [ ]:
food_trimmed.hist("P344pr", bins = 100)
plt.xlabel("Weekly income (£)")
plt.ylabel("Frequency")

If we arrange these in order and take the middle point we obtain the median income:

In [ ]:
food_trimmed["P344pr"].median()

If your data have an even number of items, the median is the mean (average) of the two middle points.
For example, using the following example data with four data points - 2, 4, 6, 8 - there is no one middle point.
Instead 4 and 6 are the middle points.
The median is the mean of these, which is $\frac{(4 + 6)}{2} = 5$.

The median is often considered more *robust* than the mean, which means it is less susceptible to outliers, for reasons we'll get to in a moment.


## Mean

The mean is what most people think of when they think of an average.
You simply "add them all up and divide by how many you have".
For example, the mean of the incomes (once we removed the top--coded income) is:

In [ ]:
food_trimmed.P344pr.mean()

If the incomes were an ideal normal distribution, the mean and the median would be identical.
In the wild, most distributions are not exactly normal (or ideal) so the mean and the median differ, as we have seen with our example data.

If there are outliers in our data set these can affect the mean up or down.
For example, if there are a few individuals in our data that are substantially wealthier than most this can affect the mean.
The mean of the untrimmed data (i.e. with the top--coded individuals still included) is:

In [ ]:
food.P344pr.mean()

These individuals have increased the mean, but do not affect the median as substantially:

In [ ]:
food.P344pr.median()

For this reason we often consider the median a more *robust* measure of central tendency than the mean, and why you should be careful when someone presents a mean value without any additional information.
For comparison, a distribution of the untrimmed data looks like this:

In [ ]:
food.hist("P344pr", bins = 100)
plt.xlabel("Weekly income (£)")
plt.ylabel("Frequency")

# Measures of spread

We saw in the measures of central tendency chapter that the mean can be a poor representation of data if the data is skewed, and that we should therefore be careful when someone presents us with a mean (or average) without any further information.

One of the types of 'further information' that can help us is a measure of spread of the data around the mean value.
We usually use the *variance* and the *standard deviation* to quantify measure of spread.
Both are easy to calculate, and even easier to convert between each other.


## Variance

To calculate the variance:

1. subtract the mean from each score
1. square the result
1. sum the results to produce one value
1. divide by $n - 1$ (number of observations minus one)

$$
\frac{\Sigma (x - \bar{x}) ^ 2}{n-1}
$$

Using $n - 1$ rather than simply the number of observations is known as [Bessel's correction](https://en.wikipedia.org/wiki/Bessel%27s_correction).
To calculate the variance of the population we must assume that the population mean is the same as the sample mean that we have observed.
In fixing the population mean we reduce the degrees of freedom of our observations, because if we change these the final observation is determined in order for the mean to remain constant.

For example if our sample mean is 100 we assume our population mean is also 100.
If we have two observations these might be 110 and 90 (mean 100).
If we change the 110 value to 120, the 90 value *must* change to 80 to ensure the sample mean (and therefore the population mean) remains 100, so there is only one degree of freedom.
We would therefore use $2 - 1$ as the denominator in our variance calculation.

The variance of our trimmed income data is:

In [ ]:
food_trimmed["P344pr"].var()

## Standard deviation

As you've probably noticed the variance is not in the units of the original data.
This is where the standard deviation comes in.
In fact the unit of the variance is the *square* of the unit of the original data.
The standard deviation is therefore a measure of spread in the units of the original data, and is calculated simply by square rooting the variance:

$$
\sqrt{\frac{\Sigma (x - \bar{x}) ^ 2}{n-1}}
$$

The standard deviation of the trimmed income is therefore:

In [ ]:
food_trimmed.P344pr.std()

In [ ]:
food_trimmed.P344pr.mean() - 1.96 * food_trimmed.P344pr.std()

and:

In [ ]:
food_trimmed.P344pr.mean() + 1.96 * food_trimmed.P344pr.std()

This is most of the data set, so the variability of the data suggest the mean is less useful than the median.

# Standard error and confidence intervals

You can calculate a standard error of many parameters, but typically it refers to the standard error of the mean ($SE_{\bar{x}}$).
The standard error of the mean differs from the standard deviation. The standard deviation quantifies how well the sample mean fits the observed (i.e. sample) data, and the standard error of the mean quantifies how well the sample mean matches the population mean.

Because any sample we take from the population is going to be slightly different from all other samples (because everything varies) each sample mean is going to be slightly different from every other.
The standard error of the mean is a measure of how confident our sample mean matches the population mean.

One approach to calculate the standard error of the mean would be to take multiple samples.
The mean of each of these samples would form a sampling distribution due to variation: some sample means would be lower than the population mean; some sample means would be higher than the population mean; and many would be the same.
These sample mean values would form a normal distribution around the population mean.
The standard deviation of these sample means would tell us how well our sample means fit the population mean.

In practice we can usually only take one sample so we can estimate it with:

$$
\sigma_{\bar{x}} = \frac{s}{\sqrt{n}}
$$

where $\sigma_{\bar{x}}$ is the standard error of the population mean, $s$ is the sample standard deviation, and $n$ is the number of observations in the sample.

We can demonstrate this with the census (in fact, we could demonstrate this with any data set and pretend it's the population and take multiple samples from it, but why not just use an actual population?).
I'm using ages of all people in Sheffield in 2011 to illustrate this, which I download from Nomisweb:

In [ ]:
if not os.path.isfile("../data/external/age.csv"):
    age = "https://www.nomisweb.co.uk/api/v01/dataset/NM_503_1.data.csv?date=latest&geography=1946157123&rural_urban=0&c_age=1...101&measures=20100&signature=NPK-0c73734c0f725c979cee3a:0xa9b892a105be9e9449cdb6c88bdac678e12b229e"
    age = requests.get(age)
    age = age.text
    outfile = open("../data/external/age.csv", "w")
    outfile.write(age)
    
age = pd.read_csv("../data/external/age.csv")
age = age[["C_AGE_NAME", "OBS_VALUE"]]
age.C_AGE_NAME = age.C_AGE_NAME.str.replace("Age under 1", "0")
age.C_AGE_NAME = age.C_AGE_NAME.str.replace(" and over", "")
age.C_AGE_NAME = age.C_AGE_NAME.str.replace("Age ", "")
age.C_AGE_NAME = pd.to_numeric(age.C_AGE_NAME)
sum_obsvalue = age.OBS_VALUE.sum()  # for checks

age = age.loc[age.index.repeat(age["OBS_VALUE"])]

if len(age.index) != sum_obsvalue:
    raise Exception("age data frame not spread correctly")

age.describe()

The following is a histogram of ages of people living in Sheffield from the 2011 Census.

In [ ]:
age.hist("C_AGE_NAME", bins = 100)
plt.xlabel("Age")
plt.ylabel("Frequency")

The median age is:

In [ ]:
age.C_AGE_NAME.median()

The mean age is (remember this is the mean of the population, which we wouldn't normally know):

In [ ]:
age.C_AGE_NAME.mean()

Let's take 1000 samples of 100 people from the population, and make a sampling distribution of these means:

In [ ]:
samples = []

for i in range(1000):
    sample = age.sample(n = 100, replace = True, random_state = i)
    sample = sample.C_AGE_NAME.mean()
    samples.append(sample)

samples = pd.Series(samples)

The following is a distribution of the sample means (a sampling distribution):

In [ ]:
samples.hist(bins = 20)
plt.xlabel("Mean age")
plt.ylabel("Frequency")

Most sample means are around 38, although a few are as low as 32 and as high as 45. Remember we know the population mean, but we wouldn't normally know this.
If we just had access to one sample, how would we know if the resultant sample mean was close to the population mean?

From the histogram of sample means we can see that it's more likely to end up with a sample mean that's close to the population mean than one that's further away, and we can quantify this with a confidence interval.

Let's take one sample of 1000 random cases from the original data set and pretend it's all we have access to:

In [ ]:
sample = age.sample(n = 1000, replace = True, random_state = 42)

The mean of this sample is:

In [ ]:
sample.C_AGE_NAME.mean()

This is very close to the population mean but not quite the same.
The standard error of this sample mean is:

In [ ]:
sample.C_AGE_NAME.sem()

We know that an ideal normal distribution will have [95\% of cases within 1.96 standard deviations of the mean](https://en.wikipedia.org/wiki/Normal_distribution#Standard_deviation_and_coverage).
If we multiply our standard error by $\pm$ 1.96 that therefore defines a 95\% confidence interval.
In this case we would have an interval of:

In [ ]:
sample.C_AGE_NAME.mean() - 1.96 * sample.C_AGE_NAME.sem()

to:

In [ ]:
sample.C_AGE_NAME.mean() + 1.96 * sample.C_AGE_NAME.sem()

This effectively estimates that if we were to take 100 samples the population mean would fall within these bounds 95 times.
In our example the population mean is indeed within the 95\% confidence interval of the sample mean (remember the population mean was:)

In [ ]:
age.C_AGE_NAME.mean()

To calculate 99\% confidence intervals use 2.58 standard deviations rather than 1.96, in this example resulting in a confidence interval between:

In [ ]:
sample.C_AGE_NAME.mean() - 2.58 * sample.C_AGE_NAME.sem()

and:

In [ ]:
sample.C_AGE_NAME.mean() + 2.58 * sample.C_AGE_NAME.sem()

Perhaps counterintuitively this results in a wider interval (because the interval ensures the population mean falls within these bounds 99 times instead of 95); it is not 'more precise'.

From the standard error of the mean and confidence interval we can therefore quantify how confident we are that the sample mean is close to the true population mean.

## Comparing means and confidence intervals

A useful property of confidence intervals is that they can be used to compare two or more means to see if they are statistically significantly different.
For example, we have a sample of the ages of people in Sheffield and we could create a similar sample of the ages of people in Eastbourne, calculate the confidence intervals, and compare them to see if they differ (i.e. if, on average, people are older in Eastbourne).

In [ ]:
if not os.path.isfile("../data/external/age_eb.csv"):
    age_eb = "https://www.nomisweb.co.uk/api/v01/dataset/NM_1531_1.data.csv?date=latest&geography=1946157295&c_age=1...101&measures=20100&signature=NPK-0c73734c0f725c979cee3a:0x65c03934cecfca562500f65e9cb1bd2083fbea01"
    age_eb = requests.get(age_eb)
    age_eb = age_eb.text
    outfile = open("../data/external/age_eb.csv", "w")
    outfile.write(age_eb)
    
age_eb = pd.read_csv("../data/external/age_eb.csv")
age_eb = age_eb[["C_AGE_NAME", "OBS_VALUE"]]
age_eb.C_AGE_NAME = age_eb.C_AGE_NAME.str.replace("Age under 1", "0")
age_eb.C_AGE_NAME = age_eb.C_AGE_NAME.str.replace(" and over", "")
age_eb.C_AGE_NAME = age_eb.C_AGE_NAME.str.replace("Age ", "")
age_eb.C_AGE_NAME = pd.to_numeric(age_eb.C_AGE_NAME)
sum_obsvalue = age_eb.OBS_VALUE.sum()  # for checks

age_eb = age_eb.loc[age_eb.index.repeat(age_eb["OBS_VALUE"])]

if len(age_eb.index) != sum_obsvalue:
    raise Exception("age_eb data frame not spread correctly")

age_eb.describe()

From the table above we can see that the mean *is* higher than the mean for Sheffield, but remember we are comparing populations and we would not typically have access to this information.
Let's take a sample of 1,000 individuals and calculate the mean and standard error of the mean:

In [ ]:
sample_eb = age_eb.sample(n = 1000, replace = True, random_state = 42)
sample_eb.C_AGE_NAME.mean()

So our sample mean is also different, but can we be sure it's different and not just the result of our sampling?'. Using the standard error of the mean our confidence interval is:

In [ ]:
sample_eb.C_AGE_NAME.mean() - 1.96 * sample_eb.C_AGE_NAME.sem()

and

In [ ]:
sample_eb.C_AGE_NAME.mean() + 1.96 * sample_eb.C_AGE_NAME.sem()

Note that, reassuringly, this interval contains our population mean. Note also that the full interval is higher than the Sheffield interval (i.e. the top of the Sheffield interval is below the bottom of the Eastbourne interval). This means that the mean age of Sheffield and Eastbourne are statistically significantly different at the 95% confidence level.

We can visualise the confidence intervals as:

In [ ]:
age_cis = {"town": ["Sheffield", "Eastbourne"],
           "mean": [sample.C_AGE_NAME.mean(), sample_eb.C_AGE_NAME.mean()],
           "sem":  [sample.C_AGE_NAME.sem(), sample_eb.C_AGE_NAME.sem()]}

age_cis = pd.DataFrame.from_dict(age_cis)
age_cis

In [ ]:
plt.errorbar(
    x = [1, 2], y = age_cis["mean"],
    xerr = 0.02, yerr = 1.96 * age_cis["sem"],
    linestyle = ""
)
plt.xticks([1, 2], age_cis["town"])
plt.ylabel("Mean age")
plt.xlabel("Town")

# Normal distribution

The normal distribution is central to our ability to infer about a population from a sample. The normal distribution looks like this (by Dan Kernler from Wikimedia Commons, CC BY-SA 4.0):

<a title="By Dan Kernler [CC BY-SA 4.0 
 (https://creativecommons.org/licenses/by-sa/4.0
)], from Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Empirical_Rule.PNG"><img width="512" alt="Empirical Rule" src="https://upload.wikimedia.org/wikipedia/commons/a/a9/Empirical_Rule.PNG"></a>

The normal distribution was discovered by Gauss (which is why it's also sometimes called the Gaussian distribution) and described an 'ideal' situation. Lots of data follow this pattern: the height or weight of a population;  The mean of the normal distribution ($\mu$ on the diagram above) is the centre. Because the normal distribution is **symmetrical** we know that 50% of cases fall below and 50% of cases fall above the mean.

Another useful property of the normal distribution is we know, or can calculate, how many cases fall with 1, 2, 3, or more standard deviations of the mean (these are shown as $\mu \pm \sigma; \mu \pm 2\sigma; \mu \pm 3\sigma$ on the figure). These are about 68%, 95%, and 97.5% respectively.

|Number of $\sigma$ from mean | Percent of cases|
|-----------------------------|-----------------|
| 1                           | 68.27%          |
| 1.96                        | 95%             |
| 2                           | 95.45%          |
| 2.58                        | 99%             |
| 3                           | 99.73%          |

Therefore if we know the mean ($\mu$) and standard deviation ($\sigma$) of our sample we can calculate the confidence interval of our sample mean (as we did above). Typically we calculate a 95% confidence interval (although 99% is also common), and this tells us the likely range the population mean falls within. This is why we used the figure 1.96 when calculating our confidence interval earlier, and this is the property that allows us to infer information about the population from our smaller sample.

The other, related, way we can use this information is if we have a mean and standard deviation and observe a data point, we can calculate how many standard deviations from the mean this data point is. We can then see if the observed data point falls within the normal variation we expect (i.e. within 1.96 standard deviations for 95% confidence) or outside it, and is therefore the result of something not within the normal range.

Remember our household weekly income example, after we removed the top--coded responses? It looked something like this:

In [ ]:
food_trimmed.P344pr.hist(bins = 100)
plt.xlabel("Weekly income (£)")
plt.ylabel("Frequency")

The mean income is:

In [ ]:
food_trimmed.P344pr.mean()

and the standard deviation is:

In [ ]:
food_trimmed.P344pr.std()

Now let's imagine we find a respondent who lives in Chelsea and we ask them to complete our survey. They respond that their income is £2,000 per week. Does this fall within the variability we expect, or is it statistically significantly likely that this respondent has a higher income than most? Well, we can calculate how many standard deviations our observed data point is away from the mean. We know:

$$
2000 = \mu + x.\sigma
$$

where $\mu$ is the mean and $x.\sigma$ is the number of standard deviations we want to calculate. If we rearrange the equation we get:

$$
\frac{2000}{\sigma} = \frac{\mu}{\sigma} + x
$$

$$
\frac{(2000 - \mu)}{\sigma} = x
$$

Plugging in the mean and standard deviation we get:

In [ ]:
(2000 - food_trimmed.P344pr.mean()) / food_trimmed.P344pr.std()

So our observed data point is more than five standard deviations higher than the mean, which means that if we were to interview 3.5 million people, only one would have an income that high. It's therefore highly likely that this respondent has a higher income than the average. In physics this would be known as a 'five-sigma' result: i.e. the result is more than five standard deviations ($\sigma$) from the mean and is therefore highly unlikely to be through chance alone (in the social sciences we usually opt for the '1.96 sigma' rule).

Not all observed data form a perfect normal distribution (in fact most differ at least slightly). There are two ways we need to describe a distribution if it is different from the normal: skewness and kurtosis.

## Skewness

A normal distribution has its mean, median, and mode at the same point (the centre). Skewness means the data points are skewed one way or the other:

<a title="By Rodolfo Hermans (Godot) at en.wikipedia. [CC BY-SA 3.0 
 (https://creativecommons.org/licenses/by-sa/3.0
)], from Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Negative_and_positive_skew_diagrams_(English).svg"><img width="500" alt="Negative and positive skew diagrams (English)" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Negative_and_positive_skew_diagrams_%28English%29.svg/500px-Negative_and_positive_skew_diagrams_%28English%29.svg.png"></a>

Negative skew means the tail is to the left; positive skew means the tail is to the right. In a positively skewed distribution the mode and median are lower than the mean. In a negatively skewed distribution the median and mode are higher than the mean.

## Kurtosis

Kurtosis refers to how bunched (clustered) around the mean the data points are. Positive kurtosis (leptokurtic) means the points are clustered around the mean, making the distribution narrower and taller than a normal distribution. Negative kurtosis (platykurtic) means the data points are spread out more widely, resulting in a distribution that is flatter and broader than a normal distribution.

In the figure below:

- the **blue** line is a normal distribution,
- the **green** line is a distribution with positive kurtosis (leptokurtic)
- the **orange** line is a distribution with negative kurtosis (platykurtic)

In [ ]:
mu = 0
variance = 1
sigma = math.sqrt(variance)
normal = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 100)
plt.plot(normal, scipy.stats.norm.pdf(normal, mu, sigma))

variance = 2
sigma = math.sqrt(variance)
platykurtic = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 100)
plt.plot(platykurtic, scipy.stats.norm.pdf(platykurtic, mu, sigma))

variance = 0.5
sigma = math.sqrt(variance)
leptokurtic = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 100)
plt.plot(leptokurtic, scipy.stats.norm.pdf(leptokurtic, mu, sigma))

plt.show()